In [1]:
import random
import numpy as np
import yfinance as yf
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import norm
import math
import scipy.stats as stats
from datetime import datetime, timedelta
%matplotlib inline

In [2]:
tickers=['AAPL','CAT','CVX','JPM','WMT']

In [3]:
df=yf.download(tickers,start='2011-12-30',end='2022-12-31')['Close']

[*********************100%***********************]  5 of 5 completed


In [4]:
df

,AAPL,CAT,CVX,JPM,WMT
Date,,,,,
2011-12-29,14.468571,90.580002,107.470001,33.419998,59.990002
2011-12-30,14.464286,90.599998,106.400002,33.250000,59.759998
2012-01-03,14.686786,93.980003,110.370003,34.980000,60.330002
2012-01-04,14.765714,94.849998,110.180000,34.950001,59.709999
2012-01-05,14.929643,95.519997,109.099998,35.680000,59.419998
...,...,...,...,...,...
2022-12-23,131.860001,239.869995,177.399994,131.279999,143.770004
2022-12-27,130.029999,243.139999,179.630005,131.740005,143.809998
2022-12-28,126.040001,239.380005,176.979996,132.460007,141.289993


In [5]:
pval=10000
port_shares=np.array([(pval/len(df.columns))/df[df.columns[i]][0] for i in range(len(df.columns))])
port_shares

array([138.23065458,  22.0799289 ,  18.6098444 ,  59.84440783,
        33.33888888])

In [6]:
df1=port_shares*df
df1['Total']=df1.sum(axis=1)
for i in df.index:
    df1.loc[i,['AAPL','CAT','CVX','JPM','WMT','Total']]=df1.loc[i,['AAPL','CAT','CVX','JPM','WMT','Total']].values/sum(df1.loc[i,['AAPL','CAT','CVX','JPM','WMT']].values)

In [7]:
df1['Volatility']=[np.nan for i in range(len(df1))]
df1

,AAPL,CAT,CVX,JPM,WMT,Total,Volatility
Date,,,,,,,
2011-12-29,0.200000,0.200000,0.200000,0.200000,0.200000,1.0,NaN
2011-12-30,0.200702,0.200805,0.198762,0.199740,0.199991,1.0,NaN
2012-01-03,0.197797,0.202172,0.200116,0.203953,0.195962,1.0,NaN
2012-01-04,0.198780,0.203962,0.199691,0.203697,0.193870,1.0,NaN
2012-01-05,0.199986,0.204380,0.196749,0.206916,0.191968,1.0,NaN
...,...,...,...,...,...,...,...
2022-12-23,0.461746,0.134171,0.083634,0.199025,0.121424,1.0,NaN
2022-12-27,0.456615,0.136382,0.084923,0.200283,0.121799,1.0,NaN
2022-12-28,0.450906,0.136791,0.085239,0.205155,0.121909,1.0,NaN


In [8]:
rebalance=pd.DataFrame(columns=['AAPL','CAT','CVX','JPM','WMT'],index=df1.index)
rebalance

,AAPL,CAT,CVX,JPM,WMT
Date,,,,,
2011-12-29,NaN,NaN,NaN,NaN,NaN
2011-12-30,NaN,NaN,NaN,NaN,NaN
2012-01-03,NaN,NaN,NaN,NaN,NaN
2012-01-04,NaN,NaN,NaN,NaN,NaN
2012-01-05,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2022-12-23,NaN,NaN,NaN,NaN,NaN
2022-12-27,NaN,NaN,NaN,NaN,NaN
2022-12-28,NaN,NaN,NaN,NaN,NaN


In [9]:
for i in df1.index:
    vol=sum(abs(df1.loc[i,['AAPL','CAT','CVX','JPM','WMT']].values-df1.loc['2011-12-29',['AAPL','CAT','CVX','JPM','WMT']].values))
    df1.loc[i,'Volatility']=vol

In [10]:
df1

,AAPL,CAT,CVX,JPM,WMT,Total,Volatility
Date,,,,,,,
2011-12-29,0.200000,0.200000,0.200000,0.200000,0.200000,1.0,0.000000
2011-12-30,0.200702,0.200805,0.198762,0.199740,0.199991,1.0,0.003014
2012-01-03,0.197797,0.202172,0.200116,0.203953,0.195962,1.0,0.012483
2012-01-04,0.198780,0.203962,0.199691,0.203697,0.193870,1.0,0.015317
2012-01-05,0.199986,0.204380,0.196749,0.206916,0.191968,1.0,0.022592
...,...,...,...,...,...,...,...
2022-12-23,0.461746,0.134171,0.083634,0.199025,0.121424,1.0,0.523492
2022-12-27,0.456615,0.136382,0.084923,0.200283,0.121799,1.0,0.513794
2022-12-28,0.450906,0.136791,0.085239,0.205155,0.121909,1.0,0.512121


In [11]:
df1[df1['Volatility']>0.1]

,AAPL,CAT,CVX,JPM,WMT,Total,Volatility
Date,,,,,,,
2012-02-22,0.223697,0.225843,0.176691,0.201219,0.172549,1.0,0.101520
2012-02-23,0.223891,0.225328,0.177086,0.202294,0.171402,1.0,0.103025
2012-02-24,0.225813,0.224258,0.177738,0.200580,0.171611,1.0,0.101302
2012-02-27,0.226186,0.222485,0.177789,0.203699,0.169841,1.0,0.104740
2012-02-28,0.228846,0.221293,0.176606,0.203157,0.170098,1.0,0.106593
...,...,...,...,...,...,...,...
2022-12-23,0.461746,0.134171,0.083634,0.199025,0.121424,1.0,0.523492
2022-12-27,0.456615,0.136382,0.084923,0.200283,0.121799,1.0,0.513794
2022-12-28,0.450906,0.136791,0.085239,0.205155,0.121909,1.0,0.512121


In [12]:
#we will rebalace our portfolio when The volatility excedes 10% 